In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import h5py
from pathlib import Path
import json
from collections import defaultdict
import pymeshlab

In [ ]:
chair_dir = Path("/home/pauldelseith/HyperDiffusion_semantic_decomposition/data/chair/3068")

In [ ]:
with open(chair_dir / "result_after_merging.json") as f:
    meta = json.load(f)

In [ ]:
ms = pymeshlab.MeshSet()
ms.load_new_mesh(str(chair_dir / "objs" / "original-1.obj"))

In [ ]:
def load_h5_file(file_path):
    with h5py.File(file_path, 'r') as h5_file:
        # List all groups
        print("Keys: %s" % h5_file.keys())
        # Get the data
        data = {}
        for key in h5_file.keys():
            data[key] = h5_file[key][()]
        return data

In [ ]:
parent_dir = Path("/home/pauldelseith/dataset_storage/partnet/data_v0/")
model_class_anno_id_mapping = defaultdict(list)
for folder in parent_dir.iterdir():
    if folder.is_dir():
        with open(folder / "meta.json", "r") as f:
            meta = json.load(f)
        model_class_anno_id_mapping[meta["model_cat"]].append(meta["anno_id"])

In [ ]:
with open(Path("partnet_class_mapping.json"), "w") as f:
    json.dump(model_class_anno_id_mapping, f)

In [ ]:
import os
os.getcwd()

In [ ]:
sem_seg_path = Path("/home/pauldelseith/dataset_storage/partnet/sem_seg_h5/Chair-1/train-00.h5")

In [ ]:
ins_seg_path = Path("/home/pauldelseith/dataset_storage/partnet/ins_seg_h5/Chair/train-00.h5")

In [ ]:
shapenet_seg = Path("/home/pauldelseith/dataset_storage/shapenet_seg/hdf5_data/ply_data_train0.h5")

In [ ]:
sem_seg_annotations = load_h5_file(sem_seg_path)

In [ ]:
ins_seg_annotations = load_h5_file(ins_seg_path)

In [ ]:
shapenet_seg_h5 = load_h5_file(shapenet_seg)

In [ ]:
ins_seg_annotations["nor"].shape

In [ ]:
sem_seg_annotations.keys()

In [ ]:
shapenet_seg_h5.keys()

In [ ]:
shapenet_seg_h5["pid"]

In [ ]:
import trimesh

In [ ]:
sem_seg_annotations["data"].shape

In [ ]:
pc = trimesh.points.PointCloud(sem_seg_annotations["data"][0])

In [ ]:
pc_convex_hull = pc.convex_hull

In [ ]:
sem_seg_annotations["data"].shape

In [ ]:
sem_seg_points = sem_seg_annotations["data"][0]

In [ ]:
npy_file_path = "/home/pauldelseith/HyperDiffusion_semantic_decomposition/data/partnet_100000_pc_occ_in_out_True/new-1_manifold.obj.npy"
point_cloud_data = np.load(npy_file_path)

In [ ]:
point_cloud_data
point_cloud_data.shape
x, y, z, intensity = point_cloud_data[point_cloud_data[:,3] == 1].T
colors = intensity

In [ ]:
colors.min()

In [ ]:
colors.max()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
scatter = ax.scatter(x, y, z, c=colors, cmap='viridis', marker='o')
scatter.axes.set_xlim3d([-1,1]) 
scatter.axes.set_ylim3d([-1,1]) 
scatter.axes.set_zlim3d([-1,1])
plt.show()

In [ ]:
import trimesh

In [ ]:
mesh = trimesh.load("/home/pauldelseith/HyperDiffusion_semantic_decomposition/data/partnet/new-1_manifold.obj")

In [ ]:
mesh.vertices.max()

In [ ]:
mesh.vertices.min()

In [ ]:
point_cloud_data = mesh.sample(100000)

In [ ]:
points_surface = mesh.sample(100000) + 0.01 * np.random.randn(100000, 3)
pc_export = trimesh.PointCloud(points_surface).export("pc1.obj")

In [ ]:
import open3d as o3d
import numpy as np
def visualize_npy_pointcloud(file_path):
    # Load the point cloud data from the .npy file
    point_cloud_data = np.load(file_path)
    point_cloud_data = point_cloud_data[point_cloud_data[:,3] == 1] # filter 1s
    # Create two Open3D PointCloud objects for different markers
    pcd_0 = o3d.geometry.PointCloud()
    pcd_1 = o3d.geometry.PointCloud()
    # Separate points based on the fourth parameter
    mask_0 = point_cloud_data[:, 3] == 0
    mask_1 = point_cloud_data[:, 3] == 1
    points_0 = point_cloud_data[mask_0][:, :3]
    points_1 = point_cloud_data[mask_1][:, :3]
    # Assign the numpy point cloud data to the Open3D PointClouds
    pcd_0.points = o3d.utility.Vector3dVector(points_0)
    pcd_1.points = o3d.utility.Vector3dVector(points_1)
    # If the point cloud has colors (assuming RGB format with values 0-255)
    if point_cloud_data.shape[1] == 7:
        colors_0 = point_cloud_data[mask_0][:, 4:7] / 255.0
        colors_1 = point_cloud_data[mask_1][:, 4:7] / 255.0
        pcd_0.colors = o3d.utility.Vector3dVector(colors_0)
        pcd_1.colors = o3d.utility.Vector3dVector(colors_1)
    else:
        # Assign default colors
        pcd_0.paint_uniform_color([1, 0, 0])  # Red for points with 4th parameter 0
        pcd_1.paint_uniform_color([0, 1, 0])  # Green for points with 4th parameter 1
    # Visualize the point clouds
    o3d.visualization.draw_plotly([pcd_0, pcd_1])
# Example usage
visualize_npy_pointcloud("/home/pauldelseith/HyperDiffusion_semantic_decomposition/data/partnet_100000_pc_occ_in_out_True/new-11_manifold.obj.npy")

In [ ]:
point_cloud_data = ins_seg_annotations["pts"]
#point_cloud_data = np.concatenate((point_cloud_data, sem_seg_annotations["label_seg"][:,:,np.newaxis]), axis=-1)

In [ ]:
list(sem_seg_annotations.keys())

In [ ]:
sem_seg_annotations["label_seg"].shape

In [ ]:
sem_seg_annotations["label_seg"].min()

In [ ]:
x, y, z, intensity = point_cloud_data[0].T
colors = intensity

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
scatter = ax.scatter(x, y, z, c=colors, cmap='viridis', marker='o')
scatter.axes.set_xlim3d([-1,1]) 
scatter.axes.set_ylim3d([-1,1]) 
scatter.axes.set_zlim3d([-1,1])
plt.show()

In [ ]:
import igl

In [ ]:
n_points_uniform = 10000

In [ ]:
points_uniform = np.random.uniform(
                    -1.0, 1.0, size=(n_points_uniform, 3)
                )
points_surface = point_cloud_data[0][:,:3]
points_surface += 0.01 * np.random.randn(points_surface.shape[0], 3)

In [ ]:
points = np.concatenate([points_surface, points_uniform], axis=0)